In [1]:
import requests
import json
import jwt
import base64
from itertools import product
from jsonschema import Draft202012Validator
import pandas as pd
import secrets 
import re
import os

from rembg import remove
from io import BytesIO
from PIL import Image
from requests import get
import time

## validator

In [2]:

specficationSchema = {
    "type": "object",
                 "patternProperties": {"^[a-zA-Z_][a-zA-Z0-9_]*$": {"type": "string"}}
                 }
varientsSchema = {
    "type": "object",
                 "properties": {
                                "color": {"type": "array", "minItems": 1, "items": {"type": "string"}},
                                "size": {"type": "array", "minItems": 1, "items": {"type": "string"}},                              
                                },
                                "required": ["color","size"],
                                "additionalProperties": False   
                 }

inventorySchema = {
                    "type": "array", "minItems": 1, "items": {"type": "object","properties": {
                        "patternProperties": {
                                "^[a-zA-Z_#][a-zA-Z0-9_#]*$": {"type": "string"}
                            },
                            "properties": {
                                "active": {"type": "boolean"}
                            },
                            "required": ["active"],
                            "additionalProperties": False    
                            
                    }}
}

variation_overrideSchema = {"type": "array"}
locationIdSchema = {"type": "string"}

specfications_validator = Draft202012Validator(specficationSchema)
varients_validator = Draft202012Validator(varientsSchema)
inventory_validator = Draft202012Validator(inventorySchema)
locationId_validator = Draft202012Validator(locationIdSchema)
variation_override_validator = Draft202012Validator(variation_overrideSchema)



## decode token

In [3]:
def decode_jwt_without_secret(token):
    try:
        # Split the token into its parts
        header, payload, signature = token.split('.')

        # JWT payloads are base64url encoded, but they might not have the right length,
        # so we pad them with '=' until they do.
        missing_padding = len(payload) % 4
        if missing_padding != 0:
            payload += '=' * (4 - missing_padding)

        # Decode the base64url encoded header and payload
        decoded_header = base64.urlsafe_b64decode(header + '==').decode('utf-8')
        decoded_payload = base64.urlsafe_b64decode(payload + '==').decode('utf-8')

        return {
            'header': decoded_header,
            'payload': decoded_payload
        }
    except Exception as e:
        return {'error': str(e)}

## get location id

In [4]:
def getLoctaionId(jwt_token):
    decoded_jwt = decode_jwt_without_secret(jwt_token)
    locationId = json.loads(decoded_jwt['payload'])['user']['locations'][0]['location_id']
    return locationId

## get static data

In [23]:
catalogMasterdataFile=open('../GetMasterData/catalog-full.json')
catalogMasterdata=json.load(catalogMasterdataFile)['result']

ondcDomainFile = open('../GetMasterData/ondc_domain.json')
ondcDomaindata=json.load(ondcDomainFile)['result']

attrubuteMasterdataFile = open('../GetMasterData/Attribute-2024-04-17.json')
attrubuteMasterdata=json.load(attrubuteMasterdataFile)['data']

masterColorFile = open('../GetMasterData/color.json')
masterColor = json.load(masterColorFile)['color_list']
color_list = masterColor

masterData=json.load(open('../GetMasterData/catalogs_with_attribute.json'))['data']

ondcCategoryData =pd.read_excel("../GetMasterData/OndcCategoryFix.xlsx")
incorrectData =pd.read_excel("../NewData/Bitterlime.xlsx")



## get catalog

In [24]:

def getCatalog(token):
    url = "https://asia-south1-prod-shopcircuit-seller-portal.cloudfunctions.net/api/catalog_range?startIndex=1&endIndex=10000&status=QC_PASS"
    payload = {}
    headers = {
    'authority': 'asia-south1-prod-shopcircuit-seller-portal.cloudfunctions.net',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'authorization': f'Bearer {token}',
    'if-none-match': 'W/"e7a2-+n8mrFBLKpk+whrNle/ioRtJPeA"',
    'origin': 'https://dashboard.shopcircuit.ai',
    'referer': 'https://dashboard.shopcircuit.ai/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    return json.loads(response.text)
    

## Get Nearest Color

In [7]:

def find_nearest_color(hex_color,color_list):
    def validate_color_list(color_list):
        """Validates the color list and corrects any errors."""
        valid_colors = []
        for entry in color_list:
            if not isinstance(entry, list) or len(entry) != 2:
                print("Invalid entry:", entry)
                continue  # Skip invalid entries
            color_name, hex_value = entry
            if not isinstance(color_name, str) or not isinstance(hex_value, str):
                print("Invalid entry:", entry)
                continue  # Skip invalid entries
            if not hex_value.startswith('#') or len(hex_value) != 7:
                print("Invalid hex value in entry:", entry)
                continue  # Skip invalid entries
            try:
                int(hex_value[1:], 16)  # Attempt to convert the hex value to an integer
                valid_colors.append(entry)
            except ValueError:
                print("Invalid hex value in entry:", entry)
                continue  # Skip invalid entries
        return valid_colors

    def hex_to_rgb(hex_color):
        """Converts a hexadecimal color to RGB."""
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    def rgb_distance(color1, color2):
        """Calculates the Euclidean distance between two RGB colors."""
        return sum((c1 - c2) ** 2 for c1, c2 in zip(color1, color2)) ** 0.5

    def nearest_color(hex_color, color_list):
        """Finds the nearest color in a list based on Euclidean distance in RGB space."""
        target_rgb = hex_to_rgb(hex_color)
        nearest_name = None
        nearest_hex = None
        min_distance = float('inf')
        for color_name, hex_value in color_list:
            rgb_value = hex_to_rgb(hex_value)
            distance = rgb_distance(target_rgb, rgb_value)
            if distance < min_distance:
                min_distance = distance
                nearest_name = color_name
                nearest_hex = hex_value
        return nearest_hex, nearest_name

    color_list = validate_color_list(color_list)
    hex_color = hex_color.upper()  # Ensure uppercase hex color input
    nearest_hex, nearest_name = nearest_color(hex_color, color_list)
    return nearest_hex

# hex_color = "#1E93FF"  # Example hexadecimal color
# nearest_name, nearest_hex = find_nearest_color(hex_color, color_list)
# print("Nearest color to", hex_color, "is:", nearest_name, "with hexcode:", nearest_hex)


## Get Color

In [8]:

def download_image_url(url, output_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"Error downloading image: {response.status_code}")  

def remove_background(image_path, output_path):
    with open(image_path, 'rb') as f:
        image_data = f.read()
    output = remove(image_data)
    with open(output_path, 'wb') as f:
        f.write(output)

def get_dominant_color(image_path, tolerance=10):
  img = Image.open(image_path).convert('RGBA')
  width, height = img.size
  # Create a dictionary to store pixel frequencies
  color_counts = {}
  background_color = img.getpixel((0, 0))
  # Iterate through each pixel
  for x in range(width):
    for y in range(height):
      pixel = img.getpixel((x, y))
      # Check if pixel is transparent (background)
      if pixel[3] <= tolerance:
        continue
      # Check for color similarity to background (within tolerance)
      if abs(pixel[0] - background_color[0]) <= tolerance and \
         abs(pixel[1] - background_color[1]) <= tolerance and \
         abs(pixel[2] - background_color[2]) <= tolerance:
        continue
      # Increment count for this color
      if pixel in color_counts:
        color_counts[pixel] += 1
      else:
        color_counts[pixel] = 1
  # Find the most frequent color (excluding background)
  dominant_color = max(color_counts, key=color_counts.get)
  # Convert RGB to hex code
  return f"#{dominant_color[0]:02x}{dominant_color[1]:02x}{dominant_color[2]:02x}"

def getTrueColor(hex_color):
    for item in masterColor:
        if item[1] == hex_color:
            color=item[1]
            return color
        
def getValidColor(hex_color):
    for item in masterColor:
        if item[1] == hex_color:
            valid1 =0
            return valid1
        
def matchColorName(color1):
    color = color1.lower()
    for item in masterColor:
        if item[0] == color:
            color = item[1]
            return color
        else:
            color = color.replace(" ", "")
            if item[0] == color:
                color = item[1]
                return color     
              
def getValid2ColorName(color):
    for item in masterColor:
        if item[0] == color:
            valid2=item[0]
            return valid2
        
def delete_files_in_folder(folder_path):
    try:
        for root, dirs, files in os.walk(folder_path):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

            for dir_name in dirs:
                dir_path = os.path.join(root, dir_name)
                os.rmdir(dir_path)
                print(f"Deleted directory: {dir_path}")

        print(f"All files and directories in {folder_path} deleted successfully.")
    except OSError as e:
        print(f"Error deleting files and directories: {e}")   

def getImageColor(item):
    download_path = "../CatalogOperation/downloaded_image/downloaded_image.jpg"
    output_path = "../CatalogOperation/remove_background/image_no_background.png"
    download_image_url(item, download_path)
    remove_background(download_path, output_path)

    dominant_color_hex = get_dominant_color(output_path)
    print("Color_Hex_Code --> ",dominant_color_hex)
    color = getTrueColor(dominant_color_hex) 
    colorCode = color
    valid1 = getValidColor(dominant_color_hex)           
    if valid1 == None:
        color_code = dominant_color_hex[1:]
        response = requests.get(f"https://www.thecolorapi.com/id?hex={color_code}&format=json")
        data = response.json()
        color = data['name']['value']
        color = color.lower()
        dominant_color_hex = dominant_color_hex.upper()
        color2 = matchColorName(color)
        valid2 = getValid2ColorName(color)  
        
        colorCode = dominant_color_hex
        newColor = [color,dominant_color_hex]
        if valid2 == None:
            with open('../CatalogOperation/colorNewName.txt','a') as f:
                f.write(f"\n{newColor}")
        if color2 != None:
            colorCode = color2

    print("Before Color -->",colorCode)
    colorCode =find_nearest_color(colorCode,color_list)
    print("After Color -->",colorCode)
    colorCode = colorCode[1:]      
    color= colorCode
    direc = 'catalog_operation/downloaded_image/'
    delete_files_in_folder(direc)
    return color


## update varients

In [21]:
def createInventory(color, size):
    combinations = []
    for item1, item2 in product(color, size):
        if item2 == "Free Size":
            item2 = "FreeSize"
        combinations.append({'#'.join([str(item1), str(item2)]):"1","active":True})
    return combinations


def find_color_name_code(color_name,imageUrl):
    colors = masterColor
    for color in colors:
        if color_name == color[0]:
            return f"{color[1]}".replace("#","")
    else:
        colorValue = getImageColor(imageUrl)
        return colorValue

def all_hashes(color_list):
    pattern = r'[0-9A-F]{6}\b'
    for item in color_list:
        if not re.match(pattern, item):
            return False
    return True

def updateVarients(varients_old,inventory_old,image):
    
    if 'size' in varients_old.keys():
        size = varients_old['size']
        size = list(dict.fromkeys(size))
        if size == ['Onesize'] or size == ['Pack'] :
            size = ['Free Size']
    else:
        size = ['Free Size']

    if 'color' in varients_old.keys():    
        color = getImageColor(image)
        colors = [color]
        # colors = varients_old['color']
        # colors = list(dict.fromkeys(colors))
        # colors = list(set(colors))
        # if colors[0] == 'NA' or colors[0] == 'Multicolor' or colors[0] == 'multicolor':
        #     color= getImageColor(image)
        #     colors = [color]
        # else:    
        #     color_split = [c.strip().lower() for item in colors for c in item.split('/')]
        #     color_unique = list(set(color_split))
        #     # colors=color_unique
        #     newColor=[]
        #     for colr in color_unique:
        #         newValue = matchColorName(colr)
        #         # print("The Value Changed --> ",newValue)
        #         if newValue != None:
        #             colorCode = newValue[1:]
        #             newColor.append(colorCode)    
        #     if newColor != []:
        #         color_unique = list(set(newColor))
        #     colors =  color_unique  
    else:
        color = getImageColor(image)
        colors = [color]
    
        

    if size == ['Onesize'] or size == ['Pack']:
        size = ['Free Size']
    print("colors is --> ",colors)    
    if all_hashes(colors):
        inventory = createInventory(colors,size)
        varients_old = {
            "color":colors,
            "size":size
        }
        return varients_old,inventory
    else:  
        updated_color = []
        for color in colors:
            print("for loop color -->",color)
            updated_color.append(find_color_name_code(color.lower(),image))
        inventory = createInventory(updated_color,size)

        newVarients = {
            "color":updated_color,
            "size":size
        }
        newInventory = inventory

    return newVarients, newInventory

## update catalog

In [10]:
def updateCatalog(catalog_id,data):
    admin_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyIjp7ImFkZHJlc3MiOiJMdWNraG5vdyIsInJvbGUiOiJBRE1JTiIsIm5hbWUiOiJBZG1pbiIsIm1vYmlsZSI6IjAwMDAwMDAwIiwiYWdyZWVtZW50cyI6dHJ1ZSwiY29tcGFueSI6IkFkbWluIiwiZ3BzIjoiMTIuOTExOTAwLCA3Ny42NDQ2MDAiLCJnc3RpbiI6IkFETUlOIiwiZW1haWwiOiJhZG1pbkB2aWR5YW50LmNvbSIsInN0YXR1cyI6IkFDVElWRSIsImF2YWlsYWJpbHR5Ijp7ImVuYWJsZSI6dHJ1ZSwidGltZXN0YW1wIjoiMjAyMy0wOS0yMlQxMDoxNzozNi4xNzRaIn0sInRhdCI6IlA1RCIsIm9yZGVyX21pbl92YWx1ZSI6NTAwLCJpZCI6IndrdHpxeGQzS285cFB0TjVMYTNxIiwiYXZhdGFyIjoiYXNzZXRzL2ltYWdlcy9hdmF0YXJzL21hbGUtMDcuanBnIn0sImlhdCI6MTcxMjM5MzYzOX0.zCFmfxRzBATFo6TEscXRAGPC774rVuF07pqw--JjKd0"
    url = f"http://127.0.0.1:5001/prod-shopcircuit-seller-portal/asia-south1/api/catalog/update_by_admin/{catalog_id}"
    payload = json.dumps(data)
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {admin_token}'
    }

    response = requests.request("PUT", url, headers=headers, data=payload)

    print(response.text)


## Valid Specification

In [11]:
def validSpecification(sub_sub_category_id,clothing_attributes,super_category):
    
    Resolved_attributes = {}
    for item in masterData:
        if item["sub_sub_category_id"] == sub_sub_category_id:
            for attribute in item["attribute"]:
                for attributeSeller in clothing_attributes:
                    key = attributeSeller 
                    value = clothing_attributes[attributeSeller] 

                    key1 = key.replace(' ','_')
                    key = key1.lower()

                    if key in attribute:
                        dt = attribute[key] 
                        if key == 'size' or key == 'varient' or key == 'color':
                            print(key,"-->",value)
                        elif key == 'gender':
                            print("super_category ->",super_category)
                            if 'Women Fashion' == super_category:
                                Resolved_attributes['gender'] = 'female'
                            elif 'Men Fashion' == super_category:
                                Resolved_attributes['gender'] = 'male'
                            else:
                                Resolved_attributes['gender'] = value
                        elif key == 'Brand' or key == 'brand':
                            value=value.replace("\xa0","")
                            print("Brand Name --> ",value)
                            Resolved_attributes['brand'] = value
                        # elif key == 'net_quantity_(n)':
                        #     Resolved_attributes[key] = 'Single'
                        else:
                            for item2 in dt:
                                dt1 = item2.lower()
                                dt2 = value.lower()                            
                                if dt1 in dt2:
                                    value = item2
                                    Resolved_attributes[key] = value
    return Resolved_attributes                               

## UpdateSpecification

In [12]:
def get_first_attribute_value(sub_sub_category_id, name):
    for attribute in attrubuteMasterdata:
        if attribute['data']['sub_sub_category_id'] == sub_sub_category_id and attribute['data']['name'] == name:
            if attribute['data']['validators']['required']:
                return attribute['data']['values'][0]
    return None

def get_required_attribute_values(sub_sub_category_id):
    result = {}
    
    for attribute in attrubuteMasterdata:
        if attribute['data']['sub_sub_category_id'] == sub_sub_category_id and attribute['data']['validators']['required']:
            result[attribute['data']['name']] = attribute['data']['values'][1] if len(attribute['data']['values']) > 2 else attribute['data']['values'][0]
    return result

def delete_keys_from_dict(dictionary, keys_to_delete):
    for key in keys_to_delete:
        if key in dictionary:
            del dictionary[key]
    return dictionary

def getSpecfication(sub_sub_category_id, specfication,super_category):
    specfications = validSpecification(sub_sub_category_id,specfication,super_category)
    newSpecfication = {}
    requiredValueAttribute = get_required_attribute_values(sub_sub_category_id)
    # print("specfications -->", specfications)
    # print("requiredValueAttribute --> ",requiredValueAttribute)
    for key, value in requiredValueAttribute.items():
        if key in specfications:
            if specfications[key] == None or specfications[key] == "Not Available":
                if key == "country_of_origin" or key == 'country' :
                    newSpecfication[key] = "India"
                else:
                    newSpecfication[key] = value
            else:
                if key == "country_of_origin" or key == 'country':
                    newSpecfication[key] = "India"
                else:
                    newSpecfication[key] = specfications[key]
        else:
            if key == "country_of_origin" or key == 'country':
                newSpecfication[key] = "India"
            else:
                newSpecfication[key] = value
    # print("newSpecfication --> ",newSpecfication)
    optionalSpecification = delete_keys_from_dict(specfications,list(requiredValueAttribute.keys()))
    optionalSpecification.update(newSpecfication)
    optionalSpecification = delete_keys_from_dict(optionalSpecification,["variation",'color'])
    optionalSpecification['size_chart'] = 'https://i.pinimg.com/originals/7e/b3/60/7eb360b804102c588de6de27c4962a41.jpg'
    # optionalSpecification['size_chart'] = 'https://lh3.googleusercontent.com/u/0/drive-viewer/AKGpihZ6QBVkwJTE9FB25jvTEtQwjfWPCf-FBkm_djlcZMSEM9Je1imqE49YvNgUHv_ScQzJy5xdYPP8bUURqLrcI5uCAdZ_5dFruTU=w1920-h952'
    optionalSpecification['packing-length'] = '22' 
    optionalSpecification['packing-width'] = '12' 
    optionalSpecification['packing-height'] = '5' 
    optionalSpecification['packing-weight'] = '250' 
    # print("optionalSpecification --> ",optionalSpecification)

    return optionalSpecification

##  Get Data

In [13]:
def fetchCatlog(bearer):
  id1='aL7tY99ALvY0uY2sndjk'
  url = f"http://127.0.0.1:5001/prod-shopcircuit-seller-portal/asia-south1/api/catalog/fetch_all_catalog/{id1}"

  payload = {}
  headers = {
    'Authorization': f'Bearer {bearer}'
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  data = json.loads(response.text) 
  return data 

## main execution

In [25]:
# Example JWT token
import re
target_seller_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyIjp7ImlzX3Bob25lX3ZlcmlmaWVkIjp0cnVlLCJyb2xlIjoiVVNFUiIsInNpZ25hdHVyZSI6bnVsbCwiZGVzY3JpcHRpb24iOm51bGwsImNyZWF0ZWRfYXQiOiIyMDIzLTA5LTE4VDA2OjA5OjU2LjUyOVoiLCJnc3RpbiI6IjA3QUFVQ1MzMjM1TDFaUyIsImJhbmsiOnsiaWZzY19jb2RlIjoiIiwiYWNjb3VudF9udW1iZXIiOiIiLCJiZW5lZmljaWFyeV9uYW1lIjoiIiwiYWNjb3VudF90eXBlIjoiIiwiYnJhbmNoX2FkZHJlc3MiOiIifSwiY29tcGFueV9sb2dvIjoiaHR0cHM6Ly9lbmNyeXB0ZWQtdGJuMC5nc3RhdGljLmNvbS9pbWFnZXM_cT10Ym46QU5kOUdjUkFIWFBsdXE2R3RUUlBESUhSdjVrSlB5ODZ1RmpwNXNPN2hnJnVzcXA9Q0FVIiwiaXNfZ3N0aW5fdmVyaWZpZWQiOnRydWUsImNvbXBhbnkiOiJCaXR0ZXJsaW1lIiwiY29tcGFueV91cmwiOm51bGwsInN0b3JlX3RpbWluZyI6eyJjbG9zZSI6IjE4OjMwIiwib3BlbiI6Ijk6MzAifSwiZW1haWwiOiJuaWxveS5kdXR0YUBldGhuaWNvY2Vhbi5jb20iLCJhZ3JlZW1lbnQiOnRydWUsIm1vYmlsZSI6NzgzODcwNzA4NiwiZ3BzIjoiMTIuOTExOTAwLCA3Ny42NDQ2MDAiLCJhdmFpbGFiaWx0eSI6eyJlbmFibGUiOnRydWUsInRpbWVzdGFtcCI6IjIwMjMtMDktMThUMDY6MDk6NTYuNTI5WiJ9LCJwcm9maWxlX2ltZyI6Imh0dHBzOi8vc3RhdGljLnZlY3RlZXp5LmNvbS9zeXN0ZW0vcmVzb3VyY2VzL3ByZXZpZXdzLzAwMy8yNDAvMzgzL25vbl8yeC9iZXN0LXNlbGxlci1nb2xkZW4tYmFkZ2UtaXNvbGF0ZWQtaWxsdXN0cmF0aW9uLXZlY3Rvci5qcGciLCJuYW1lIjoiTmlsb3kgRHV0dGEiLCJhZ3JlZW1lbnRfdGltZSI6IjIwMjMtMDktMThUMDY6MDk6NTYuNTI5WiIsInN0YXR1cyI6IkFDVElWRSIsInRhdCI6IlA1RCIsIm9yZGVyX21pbl92YWx1ZSI6NTAwLCJsb2NhdGlvbnMiOlt7ImFkZHJlc3MiOnsiY291bnRyeSI6IkluZGlhIiwicGluY29kZSI6MTEwMDU5LCJzdGRfY29kZSI6IjAxMSIsImNpdHkiOiJVdHRhbSBOYWdhciIsInN0cmVldCI6IjJuZCBGbG9vciBWaWxsYWdlLU1hdGlhbGEiLCJkaXN0cmljdCI6Ildlc3QgRGVsaGkiLCJsb2NhbGl0eSI6IlBsb3Qgbm8gMTkxLCBraGFzcmEgbm8gMzEvMjMiLCJzdGF0ZSI6IkRlbGhpIn0sInBpY2t1cF9sb2NhdGlvbiI6IldvcmsiLCJncHMiOiIyOC42ODEsIDc3LjI0NyIsInRpbWUiOnsic2NoZWR1bGUiOnsiaG9saWRheXMiOltdfSwicmFuZ2UiOnsic3RhcnQiOiIiLCJlbmQiOiIifSwibGFiZWwiOiJlbmFibGUiLCJ0aW1lc3RhbXAiOiIyMDI0LTAzLTA1VDExOjI3OjMwLjUwMVoifSwibG9jYXRpb25faWQiOiI1MjE4MDQ5NTJjNTA0YTViOWM5NzA3YmIwZjNhNWQ1NiJ9XSwidXBkYXRlZF9hdCI6IjIwMjQtMDMtMDVUMTE6MzA6MDQuOTQ0WiIsInNoaXBtZW50X3BpY2t1cF9sb2NhdGlvbiI6Ims5VEdsVEl3RDdYaTNQNnBVcjBUIiwiaWQiOiJrOVRHbFRJd0Q3WGkzUDZwVXIwVCIsImF2YXRhciI6ImFzc2V0cy9pbWFnZXMvYXZhdGFycy9tYWxlLTA3LmpwZyJ9LCJpYXQiOjE3MTg2MTc0MTV9.tXTStetTP4PSy-UnKTHLICFQgIU5b8lnTLMRLDCIM-s"
seller_catalog = getCatalog(target_seller_token)
# seller_catalog = fetchCatlog(target_seller_token)
result_data = []
i=0
total_length = len(seller_catalog)
for catalog in seller_catalog:
    print(f"================================================================================================================>>>{i}/{total_length} ")
    i+=1
    catalog_data = catalog["data"]
    sellerName = catalog_data["provider"]["name"]
    print("Sller Name --> ",sellerName)
    
    for index,row in incorrectData.iterrows():
        storeName = row['Store Name'] if len(str(row['Store Name']))>0 else ""
        if storeName == sellerName:
            documentId = row['ONDC Item ID'] if len(str(row['ONDC Item ID']))>0 else ""
            documentId = re.match(r'^[^_]+', documentId).group()
            if documentId == catalog["id"]:
                print("documentId -->",documentId)
                image = catalog_data["images"][0]
                catalog_id = catalog["id"]
                catalog_data["location_id"] = getLoctaionId(target_seller_token)
                catalog_data["specifications"] = getSpecfication(catalog_data["sub_sub_category_id"],catalog_data["specifications"],catalog_data["super_category"])
                newVarients, newInventory = updateVarients(catalog_data["variants"],catalog_data["inventory"],catalog_data["images"][0])
                catalog_data["variants"] = newVarients
                catalog_data["inventory"] = newInventory
                result_data.append(catalog_data)

                for index, row in ondcCategoryData.iterrows():
                    sub_sub_category2Id =row['sub_sub_category_id'] if len(str(row['sub_sub_category_id']))>0 else ""
                    # print("id 1 ",sub_sub_category2Id)
                    # print("id 2", catalog_data['sub_sub_category_id'])
                    if sub_sub_category2Id == catalog_data['sub_sub_category_id']:
                        print("Worked")
                        Ondc_category_new = row['ONDC_Category_New'] if len(str(row['ONDC_Category_New']))>0 else ""

                for item in ondcDomaindata:
                    if item['sub_sub_category_id'] == catalog_data['sub_sub_category_id']:
                        ondcDomain = item['ondc_domain']
                        print(ondcDomain)    

                price = catalog_data["ondc_price"]
                mrp = catalog_data["mrp"]
                print("mrp",mrp)
                if mrp == 0:
                    mrp=price 
                if mrp <= price:
                    mrp = price       

                data_to_update = {}
                data_to_update['location_id'] = catalog_data["location_id"]
                data_to_update['specifications'] = catalog_data["specifications"]
                data_to_update['variants'] = catalog_data["variants"]
                data_to_update['inventory'] = catalog_data["inventory"]
                data_to_update['status'] = "QC_PASS"
                data_to_update['ondc_status'] = "LIVE_ON_ONDC"
                data_to_update['cod'] = "Yes"
                data_to_update['ondc_category'] = Ondc_category_new
                data_to_update['time_to_ship'] = "P2D"
                data_to_update['order_min_value'] = 500
                data_to_update['comment'] = 'AutoPass'
                data_to_update['featuredImage'] = []
                data_to_update['mrp'] = mrp
                data_to_update['ondc_domain'] = ondcDomain
                data_to_update['prod_code'] = catalog_data["prod_code"].strip()
                data_to_update['sku_id'] = catalog_data["sku_id"].strip()

                # if 'Footwear' != catalog_data["category"]:
                #     data_to_update['status'] = "QC_IN_PROGRESS"
                #     data_to_update['ondc_status'] = "pending"

                # print(data_to_update)
                updateCatalog(catalog["id"],data_to_update)
                break
        
    
        # time.sleep(120)

        # print(str(i)+" done id => "+ catalog_id + " sku => "+catalog_data["sku_id"], data_to_update)
        
        
        # break
    
    # if i == 1 : 
    # break

# with open("resultantdatatemp.json", "w") as json_file:
#     json.dump(result_data, json_file, indent=4)
print("done")


================================================================================================================>>>0/729 
Sller Name -->  Bitterlime
================================================================================================================>>>1/729 
Sller Name -->  Bitterlime
================================================================================================================>>>2/729 
Sller Name -->  Bitterlime
================================================================================================================>>>3/729 
Sller Name -->  Bitterlime
================================================================================================================>>>4/729 
Sller Name -->  Bitterlime
================================================================================================================>>>5/729 
Sller Name -->  Bitterlime
==========================================================================================================

## testing 


In [ ]:
temp=open('bitterline draft backup.json')
temp=json.load(temp)

for i in temp: 
    print(i["id"])
    print(i['data']['variants'])
    updateCatalog(i["id"],{"variants":i['data']['variants'],'status':"QC_ERROR"})